# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
! pip install -q evaluate scikit-learn "datasets==2.15.0"

### 0. Utility Functions and Cnfigurations

In [2]:
import numpy as np
import pandas as pd

import torch
import evaluate
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

from peft import AutoPeftModelForSequenceClassification
from peft import PeftModel, PeftConfig, AutoPeftModelForSequenceClassification
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

### 1.1. Load dataset

In [3]:
# sms_spam dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [4]:
# Inspect random example.
dataset["train"][0]

{'sms': 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call\n',
 'label': 1}

### 1.2. Loading the Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

### 1.3. Loading the pre-trained HF Foundation model

In [6]:
# In this case we are doing a full fine tuning, so we will want to unfreeze all parameters.
full_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in full_model.parameters():
    param.requires_grad = True

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print(full_model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### 1.4. Evaluating Model Performance before Fine-Tuning

In [8]:
pre_trainer_model = Trainer(
    model=full_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

pre_trainer_model.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.044818,0.989238
2,No log,0.044460,0.989238
3,No log,0.049968,0.990135
4,0.038300,0.055425,0.991031
5,0.038300,0.053031,0.990135


Checkpoint destination directory ./data/spam_not_spam/checkpoint-140 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-280 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-420 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-560 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-700 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=700, training_loss=0.02793768814631871, metrics={'train_runtime': 177.3742, 'train_samples_per_second': 125.695, 'train_steps_per_second': 3.946, 'total_flos': 421188890689944.0, 'train_loss': 0.02793768814631871, 'epoch': 5.0})

In [9]:
# Evaluating the pre_trainer_model
pre_trainer_model.evaluate()

{'eval_loss': 0.04446018487215042,
 'eval_accuracy': 0.989237668161435,
 'eval_runtime': 2.7234,
 'eval_samples_per_second': 409.412,
 'eval_steps_per_second': 12.852,
 'epoch': 5.0}

In [10]:
# Make a dataframe with the predictions and the text and the labels
test_df = pd.DataFrame(tokenized_dataset["test"])
test_df = test_df[["sms", "label"]]
predictions = pre_trainer_model.predict(tokenized_dataset["test"])
test_df["predicted_label"] = np.argmax(predictions[0], axis=1)

test_df.head(10)

,sms,label,predicted_label
0,Yup... Hey then one day on fri we can ask miwa...,0,0
1,Happy new years melody!\n,0,0
2,Think I could stop by in like an hour or so? M...,0,0
3,I can make lasagna for you... vodka...\n,0,0
4,No rushing. I'm not working. I'm in school so ...,0,0
5,I only work from mon to thurs but Sat i cant l...,0,0
6,Ok... Help me ask if she's working tmr a not?\n,0,0
7,Now only i reached home. . . I am very tired n...,0,0
8,Cool. I am &lt;#&gt; inches long. hope you l...,0,0
9,"No I'm good for the movie, is it ok if I leave...",0,0


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
# Creating the PEFT Model

# Setting up configurations for PEFT
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False, 
    r=8, 
    target_modules=["q_lin", "v_lin"],
    lora_alpha=32, 
    lora_dropout=0.1
)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# creating a PEFT model
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [12]:
# Training/fine-tuning the PEFT model
peft_trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam-lora_peft_finetuning",
        # Set the learning rate
        learning_rate=1e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.040581,0.990135
2,0.044700,0.046373,0.991031
3,0.044700,0.056643,0.991031


TrainOutput(global_step=837, training_loss=0.0319589355085913, metrics={'train_runtime': 60.1647, 'train_samples_per_second': 222.34, 'train_steps_per_second': 13.912, 'total_flos': 220349720636952.0, 'train_loss': 0.0319589355085913, 'epoch': 3.0})

In [13]:
peft_trainer.evaluate()

{'eval_loss': 0.0405813604593277,
 'eval_accuracy': 0.9901345291479821,
 'eval_runtime': 2.3838,
 'eval_samples_per_second': 467.737,
 'eval_steps_per_second': 29.365,
 'epoch': 3.0}

In [14]:
peft_model.save_pretrained("./data/spam_not_spam-lora_peft_finetuned", save_adapter=True, save_config=True)
peft_model.merge_and_unload()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [15]:
test_df = pd.DataFrame(tokenized_dataset["test"])
test_df = test_df[["sms", "label"]]
predictions = peft_trainer.predict(tokenized_dataset["test"])
test_df["predicted_label"] = np.argmax(predictions[0], axis=1)
test_df.head(10)

,sms,label,predicted_label
0,Yup... Hey then one day on fri we can ask miwa...,0,0
1,Happy new years melody!\n,0,0
2,Think I could stop by in like an hour or so? M...,0,0
3,I can make lasagna for you... vodka...\n,0,0
4,No rushing. I'm not working. I'm in school so ...,0,0
5,I only work from mon to thurs but Sat i cant l...,0,0
6,Ok... Help me ask if she's working tmr a not?\n,0,0
7,Now only i reached home. . . I am very tired n...,0,0
8,Cool. I am &lt;#&gt; inches long. hope you l...,0,0
9,"No I'm good for the movie, is it ok if I leave...",0,0


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
peft_model_id = "./data/spam_not_spam-lora_peft_finetuned"
config = PeftConfig.from_pretrained(peft_model_id)
lora_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
lora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=Fals

In [18]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [19]:
sample_dataset_for_inference = tokenized_dataset["test"].select([0, 1, 22, 31, 43, 292, 448, 487])

print(sample_dataset_for_inference)

for observation in sample_dataset_for_inference:
    print(observation['input_ids'])
    print(observation)
    break

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8
})
[101, 9805, 2361, 1012, 1012, 1012, 4931, 2059, 2028, 2154, 2006, 10424, 2072, 2057, 2064, 3198, 2771, 4213, 1998, 25871, 25811, 2202, 2681, 2175, 13173, 11045, 102]
{'sms': 'Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n', 'label': 0, 'input_ids': [101, 9805, 2361, 1012, 1012, 1012, 4931, 2059, 2028, 2154, 2006, 10424, 2072, 2057, 2064, 3198, 2771, 4213, 1998, 25871, 25811, 2202, 2681, 2175, 13173, 11045, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [20]:
# Make a dataframe with the inferences and the text and the labels
items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

predictions = []
for i in items_for_manual_review:
    input_tokens = tokenizer(i['sms'], return_tensors="pt")
    #print(input_tokens)
    with torch.no_grad():
        logits = lora_model(**input_tokens).logits
        predicted_class_id = logits.argmax().item()
        predictions.append(predicted_class_id)

In [21]:
print(predictions)

[0, 0, 1, 1, 0, 1, 1, 1]


In [22]:
items_for_manual_review['label']

[0, 0, 1, 1, 0, 1, 0, 1]

## Evaluate the fine-tuned model

In [23]:
labels = [item["label"] for item in items_for_manual_review]
accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(references=labels, predictions=predictions)

print(results)

{'accuracy': 0.875}
